In [2]:
!pip install selenium

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [202]:
import pandas as pd 
import os
import re 
import numpy as np
import datetime
import sys
import settings
import json

from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome(executable_path='/Users/alexaj/anaconda3/bin/chromedriver')

In [203]:
url = "https://degreeworks.banner.marist.edu/dashboard/dashboard"
driver.get(url)

In [204]:
#driver.implicitly_wait(5)
contFrame = driver.find_element_by_css_selector('html > frameset > frame:nth-child(4)')
driver.switch_to.frame(contFrame)
#frBC = driver.find_element_by_xpath('/html/frameset')
bodyFrame = driver.find_element_by_name('frBody')
driver.switch_to.frame(bodyFrame)

In [205]:
html = driver.page_source
soup = BeautifulSoup(html, "html5lib")

In [266]:
#taking majors titles, minor titles, pathway
headTitles = soup.find_all("td", attrs={"class": "BlockHeadTitle"})
headTitle = []
for head in headTitles:
    headTitle.append(head.text)

majorArray = [x for x in headTitle if 'Major' in x]
concentrations = [x for x in headTitle if 'Concentration' in x]
minorArray = [x for x in headTitle if 'Minor' in x]
degreeName = [x for x in headTitle if 'Degree' in x]
pwayName = [x for x in headTitle if 'Pathway' in x][0]

In [267]:
#check the above variables
creditTitles = []
creditTitles = degreeName + majorArray + concentrations + minorArray 
creditTitles

['Degree in Bachelor of Science',
 'Major in Computer Science',
 'Concentration in Software Development',
 'Minor in Math']

In [272]:
jsonObjects = []

In [256]:
#code for required courses 
def req(soup):
    reqStrings = soup.find_all("td", attrs={"class": "RuleLabelTitleNeeded"})

    reqCourses = []
    for reqString in reqStrings:
        reqCourses.append(reqString.text)

    coreReqs = [x for x in reqCourses if '3 credits' in x]
    #coreReqs
    
    if not coreReqs:
        return
    else: 
        coreRTitle = []
        for x in coreReqs:
            coreRTitle.append("Core Class Required")
        coreDict = dict(zip(coreRTitle, coreReqs))

    return coreDict

# jsonObjects.append(req(soup))
# jsonObjects
jsonObjects.append(req(soup))
jsonObjects

[{'Core Class Required': '3 credits Ethics/Applied Ethics/Religious Studies'}]

In [182]:
#HONORS! Special case 
# css selector for honors frame : frmAudit > table:nth-child(38) > tbody > tr > td > table

In [259]:
def pathwayScrape(soup):
  pathwayClassesDri = driver.find_element_by_css_selector('#frmAudit > table:nth-child(28) > tbody > tr > td > table > tbody > tr:nth-child(5) > td.RuleLabelData > table')
  pathwaySoup = BeautifulSoup(pathwayClassesDri.get_attribute('innerHTML'), "html5lib")

  #In this case the exception might be different. Check if functionality for checking empty table
  pathwayTable = pathwaySoup.find_all('table')[0]
  pathwayDf = pd.read_html(str(pathwayTable))[0]
  
  pathwayDf.columns = ['Pathway No', 'Pathway Title', 'Pathway Grade', 'Pathway Credits', 'Pathway Year']

  pathwayJSON = pathwayDf.to_dict(orient="records")

  #JSON of DataFrame is default, CSV commented out
  # pathwayFileJSON = 'SerenityPathway.json'
  # pathwayJSON = pathwayDf.to_json(orient='records')
  return pathwayJSON

###################### END PWAY SCRAPE ######################

# Calling the function to get pathway  info!!
finalPathwayJSON = pathwayScrape(soup)
for x in finalPathwayJSON:
  jsonObjects.append(x)
# pathwayJSON
# pathwayFileCSV = 'SerenityPathway.csv'
# pathwayDf.to_json(pathwayFileCSV, index=False)
jsonObjects

[{'Core Class Required': '3 credits Ethics/Applied Ethics/Religious Studies'},
 {'Pathway No': 'MATH 130L',
  'Pathway Title': 'INTRO-STATIST I',
  'Pathway Grade': 'A',
  'Pathway Credits': 3,
  'Pathway Year': 'Spring 2016'},
 {'Pathway No': 'MATH 241L',
  'Pathway Title': 'CALCULUS I',
  'Pathway Grade': 'B+',
  'Pathway Credits': 4,
  'Pathway Year': 'Fall 2016'},
 {'Pathway No': 'ECON 104L',
  'Pathway Title': 'PRINC MACROECONOM',
  'Pathway Grade': 'B',
  'Pathway Credits': 3,
  'Pathway Year': 'Fall 2017'},
 {'Pathway No': 'CMPT 120L',
  'Pathway Title': 'INTRO TO PROGRAMMING',
  'Pathway Grade': 'B+',
  'Pathway Credits': 4,
  'Pathway Year': 'Spring 2016'},
 {'Pathway No': 'MATH 130L',
  'Pathway Title': 'INTRO-STATIST I',
  'Pathway Grade': 'A',
  'Pathway Credits': 3,
  'Pathway Year': 'Spring 2016'},
 {'Pathway No': 'MATH 241L',
  'Pathway Title': 'CALCULUS I',
  'Pathway Grade': 'B+',
  'Pathway Credits': 4,
  'Pathway Year': 'Fall 2016'},
 {'Pathway No': 'ECON 104L',
  'P

In [271]:
def studentInfoScrape(soup):
  studentTable = soup.find("table", attrs={"class": "Inner"})

  studentInfoDf = pd.read_html(str(studentTable))[0]

  primValue = list(studentInfoDf[0])
  primInfo = list(studentInfoDf[1])
  secondColumn = list(studentInfoDf[2])
  thirdColumn = list(studentInfoDf[3])

  finalValue = primValue + secondColumn # resulting array
  finalInfo = primInfo + thirdColumn    # resulting array
  finalInfo = [x for x in finalInfo if str(x) != 'nan']
  finalValue = [x for x in finalValue if str(x) != 'nan']
  #Vstacked dataframe if converted to a json file
    # studentInfo = np.vstack((finalValue, finalInfo)).T
    # studentInfo = studentInfo.tolist()

    # studentInfoFinalDf = pd.DataFrame(studentInfo,columns=['Values', 'Infor'])
    #studentInfoFinalDf

    # studentInfoFileJSON = 'SerenityStudentInfo.json'
  studentDict = dict(zip(finalValue, finalInfo))
  # studentJSON = json.dumps(studentDict)
    # studentJSON = studentInfoFinalDf.to_json(orient='records')
  return studentDict

################# END STUDENT VIEW SCRAPE #################

##Calling the function to get student info!!
studentInfoScrape(soup)

{'Student': 'Javellana, Alexa',
 'ID': '20068940',
 'Classification': 'Junior',
 'Advisor': 'Cannistra, Robert M',
 'Overall GPA': '3.476',
 'Student Type': 'Traditional Continuing',
 'Term Location': 'Poughkeepsie',
 'Level': 'Undergraduate',
 'Degree': 'B.S.',
 'College': 'Computer Science & Mathematics',
 'Major': 'Computer Science',
 'Minor': 'Math',
 'Concentration': 'CompSci: Software Development'}

In [32]:
semYear = datetime.date.today().year
semMonth = datetime.date.today().month
semSsn = ""

if (semMonth >= 1 and semMonth <= 5):
  semSsn = "Spring"
elif (semMonth >= 6 and semMonth <= 8):
  semSsn = "Summer"
elif (semMonth >= 9 and semMonth <= 12):
  semSsn = "Fall"
elif (semMonth == 1):
  semSsn = "Winter"

semSsn + " " + str(semYear)

'Fall 2018'

In [265]:
currClassHTML = soup.find_all("table", attrs={"class": "xBlocks"})

if(len(currClassHTML) > 2):
    currClassHTML = currClassHTML[2]
else:
    currClassHTML = currClassHTML[1]

# currClassHTML
#classNames

className = currClassHTML.find_all("td", attrs={"class": "SectionCourseTitle"})
classNames = []

for classes in className:
    classNames.append(classes.text)
    
#classNumbers

classNo = currClassHTML.find_all("td", attrs={"class": "ClassesAppliedClasses"})
classNos = []

for no in classNo:
    no = no.text.replace(u'\xa0', u' ')
    classNos.append(no)

#creditsEach

noCredits = currClassHTML.find_all("td", attrs={"class": "SectionCourseCredits"})
curCredits = []

for noCreds in noCredits:
    curCredits.append(noCreds.text)
    
# curCredits
currClassDataStack = np.vstack((classNos, classNames, curCredits)).T
currClassDataStack = currClassDataStack.tolist()

currClassInfoDf = pd.DataFrame(currClassDataStack,columns=['Current Course No', 'Current Course Title', 'Current Credit Value'])
# currClassFileCSV = 'SerenityCurrClass.csv'
# currClassInfoDf.to_csv(currClassFileCSV, index=False)

# studentInfoFileJSON = 'SerenityStudentInfo.json'
# studentView.to_json(studentInfoFileJSON)
currClassJSON = currClassInfoDf.to_dict(orient='records')
jsonObjects.append(currClassJSON)
jsonObjects

[{'Core Class Required': '3 credits Ethics/Applied Ethics/Religious Studies'},
 {'Pathway No': 'MATH 130L',
  'Pathway Title': 'INTRO-STATIST I',
  'Pathway Grade': 'A',
  'Pathway Credits': 3,
  'Pathway Year': 'Spring 2016'},
 {'Pathway No': 'MATH 241L',
  'Pathway Title': 'CALCULUS I',
  'Pathway Grade': 'B+',
  'Pathway Credits': 4,
  'Pathway Year': 'Fall 2016'},
 {'Pathway No': 'ECON 104L',
  'Pathway Title': 'PRINC MACROECONOM',
  'Pathway Grade': 'B',
  'Pathway Credits': 3,
  'Pathway Year': 'Fall 2017'},
 {'Pathway No': 'CMPT 120L',
  'Pathway Title': 'INTRO TO PROGRAMMING',
  'Pathway Grade': 'B+',
  'Pathway Credits': 4,
  'Pathway Year': 'Spring 2016'},
 {'Pathway No': 'MATH 130L',
  'Pathway Title': 'INTRO-STATIST I',
  'Pathway Grade': 'A',
  'Pathway Credits': 3,
  'Pathway Year': 'Spring 2016'},
 {'Pathway No': 'MATH 241L',
  'Pathway Title': 'CALCULUS I',
  'Pathway Grade': 'B+',
  'Pathway Credits': 4,
  'Pathway Year': 'Fall 2016'},
 {'Pathway No': 'ECON 104L',
  'P

In [20]:
# currClassesDri = driver.find_element_by_css_selector('#frmAudit > table:nth-child(67)')
# currClassSoup = BeautifulSoup(currClassesDri.get_attribute('innerHTML'), "html5lib")

currClassDataStack = np.vstack((classNos, classNames, curCredits)).T
currClassDataStack = currClassDataStack.tolist()

currClassInfoDf = pd.DataFrame(currClassDataStack,columns=['Course No', 'Course Title', 'Credit Value'])
# currClassFileCSV = 'SerenityCurrClass.csv'
# currClassInfoDf.to_csv(currClassFileCSV, index=False)

# studentInfoFileJSON = 'SerenityStudentInfo.json'
# studentView.to_json(studentInfoFileJSON)

In [274]:
#This code parses the data on DegreeWorks by td class, BlockHeadSubd
#Code parses credits into total and completed arrays
subData = soup.find_all("td", attrs={"class": "BlockHeadSubData"})
#dataNP = array of subData that is not parsed, but converted from obj to str
subDataNP = []
for subD in subData:
    subDataNP.append(subD.text)
    #objToText

#dataP = array subData that is/will be parsed
subDataP = []
for subDataN in subDataNP:
    string = subDataN.split("\xa0 ",1)[1]
    if string:
        subDataP.append(string)

    
credits = []

for subData in subDataP:
    if not any(s in subData for s in ('-', '.')):
        credits.append(subData)

    
creditTitles = []
creditTitles = degreeName + majorArray + concentrations + minorArray 
if (len(creditTitles) * 2 != len(credits)):
    creditTitles = degreeName + concentrations + minorArray 
        
        
totalCredits = []
completedCredits = []
i = 0
while i < len(credits):
    if i % 2 == 0:
        totalCredits.append(credits[i])
        i += 1
    else:
        completedCredits.append(credits[i])
        i += 1
        
creditsProgress = np.vstack((creditTitles, completedCredits, totalCredits)).T
creditsProgress = creditsProgress.tolist()
progressDf = pd.DataFrame(creditsProgress,columns=['Degree Title', 'Credits Completed', 'Total Needed'])
progressJSON = progressDf.to_dict(orient='records')

for x in progressJSON:
    jsonObjects.append(x)
jsonObjects

[{'Degree Title': 'Degree in Bachelor of Science',
  'Credits Completed': '104',
  'Total Needed': '120'},
 {'Degree Title': 'Concentration in Software Development',
  'Credits Completed': '64',
  'Total Needed': '67'},
 {'Degree Title': 'Minor in Math',
  'Credits Completed': '8',
  'Total Needed': '18'},
 {'Degree Title': 'Degree in Bachelor of Science',
  'Credits Completed': '104',
  'Total Needed': '120'},
 {'Degree Title': 'Concentration in Software Development',
  'Credits Completed': '64',
  'Total Needed': '67'},
 {'Degree Title': 'Minor in Math',
  'Credits Completed': '8',
  'Total Needed': '18'}]

In [282]:
credits

['120', '88', '73', '67', '20', '18', '22', '24', '23', '15', '23', '23']

In [170]:
#credit progress data dictionary JSON
majMinFileJSON = 'SerenityMajMin.json'
progressDf.to_json(majMinFileJSON, orient='records')

In [201]:
ppmInfo = np.vstack((ppmCountry, ppm)).T
ppmInfo = ppmInfo.tolist()

In [202]:
df = pd.DataFrame(ppmInfo,columns=['By country', 'Page views per minute'])

In [204]:
df

,By country,Page views per minute
0,GB,0.0176
1,US,0.011
2,IE,0.00347
3,DE,0.00288
4,AT,0.00278
5,BR,0.00198
6,SK,0.00169
7,HU,0.00139
8,RU,0.00129
9,VE,0.000794
